# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [27]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,accuracy_score,auc,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder

### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [3]:
df = pd.read_csv('marketing.csv')
print(df.head(5))
print("Shape of the dataset:", df.shape)
print(df.info())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumCatalogPurchases  NumStorePurchases  \
0    4/9/2012       58       635  ...                   10                  4   
1    8/3/2014       38        11  ...                    1                  2   
2  21-08-2013       26       426  ...                    2                 10   
3   10/2/2014       26        11  ...                    0                  4   
4  19-01-2014       94       173  ...                    3                  6   

   NumWebVisitsMonth  AcceptedCmp3  Accepted

### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [8]:
df = pd.read_csv('marketing.csv')
missing_percentage = (df.isna().mean() * 100).round(2)
print("Percentage of missing values:")
print(missing_percentage)

Percentage of missing values:
ID                     0.00
Year_Birth             0.00
Education              0.00
Marital_Status         0.00
Income                 1.07
Kidhome                0.00
Teenhome               0.00
Dt_Customer            0.00
Recency                0.00
MntWines               0.00
MntFruits              0.00
MntMeatProducts        0.00
MntFishProducts        0.00
MntSweetProducts       0.00
MntGoldProds           0.00
NumDealsPurchases      0.00
NumWebPurchases        0.00
NumCatalogPurchases    0.00
NumStorePurchases      0.00
NumWebVisitsMonth      0.00
AcceptedCmp3           0.00
AcceptedCmp4           0.00
AcceptedCmp5           0.00
AcceptedCmp1           0.00
AcceptedCmp2           0.00
Complain               0.00
Response               0.00
dtype: float64


In [9]:
df = pd.read_csv('marketing.csv')
df_filled = df.fillna(df.mean())
print(df_filled.head())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumCatalogPurchases  NumStorePurchases  \
0    4/9/2012       58       635  ...                   10                  4   
1    8/3/2014       38        11  ...                    1                  2   
2  21-08-2013       26       426  ...                    2                 10   
3   10/2/2014       26        11  ...                    0                  4   
4  19-01-2014       94       173  ...                    3                  6   

   NumWebVisitsMonth  AcceptedCmp3  Accepted

### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [10]:
duplicate_records = df.duplicated()
num_duplicates = duplicate_records.sum()
if num_duplicates > 0:
   
    df = df.drop_duplicates()

    print(f"{num_duplicates} duplicate record(s) found and dropped.")
else:
    print("No duplicate records found.")


print(df.head())

No duplicate records found.
     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumCatalogPurchases  NumStorePurchases  \
0    4/9/2012       58       635  ...                   10                  4   
1    8/3/2014       38        11  ...                    1                  2   
2  21-08-2013       26       426  ...                    2                 10   
3   10/2/2014       26        11  ...                    0                  4   
4  19-01-2014       94       173  ...                    3                  6   

   NumWebVisitsM

### 5. Drop the columns which you think redundant for the analysis 

In [11]:
redundant_columns = ['ID', 'Dt_Customer']
df_dropped = df.drop(redundant_columns, axis=1)
print(df_dropped.head())

   Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  Recency  \
0        1957  Graduation         Single  58138.0        0         0       58   
1        1954  Graduation         Single  46344.0        1         1       38   
2        1965  Graduation       Together  71613.0        0         0       26   
3        1984  Graduation       Together  26646.0        1         0       26   
4        1981         PhD        Married  58293.0        1         0       94   

   MntWines  MntFruits  MntMeatProducts  ...  NumCatalogPurchases  \
0       635         88              546  ...                   10   
1        11          1                6  ...                    1   
2       426         49              127  ...                    2   
3        11          4               20  ...                    0   
4       173         43              118  ...                    3   

   NumStorePurchases  NumWebVisitsMonth  AcceptedCmp3  AcceptedCmp4  \
0                  4       

### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [12]:
unique_categories = df['Marital_Status'].unique()
print("Unique Categories in 'Marital_Status':")
print(unique_categories)
df['Marital_Status'] = df['Marital_Status'].replace(['Married', 'Together'], 'relationship')
df['Marital_Status'] = df['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd'], 'Single')
unique_categories_updated = df['Marital_Status'].unique()
print("\nUnique Categories in 'Marital_Status' after Grouping:")
print(unique_categories_updated)

Unique Categories in 'Marital_Status':
['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']

Unique Categories in 'Marital_Status' after Grouping:
['Single' 'relationship']


### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [13]:
df['Total_Expenses'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
                           'MntSweetProducts', 'MntGoldProds']].sum(axis=1)
print(df.head())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation   relationship  71613.0        0         0   
3  6182        1984  Graduation   relationship  26646.0        1         0   
4  5324        1981         PhD   relationship  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumStorePurchases  NumWebVisitsMonth  \
0    4/9/2012       58       635  ...                  4                  7   
1    8/3/2014       38        11  ...                  2                  5   
2  21-08-2013       26       426  ...                 10                  4   
3   10/2/2014       26        11  ...                  4                  6   
4  19-01-2014       94       173  ...                  6                  5   

   AcceptedCmp3  AcceptedCmp4  AcceptedCmp5  AcceptedCmp

### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'

In [14]:
df['Num_Total_Purchases'] = df[['NumWebPurchases', 'NumCatalogPurchases', 
                                'NumStorePurchases', 'NumDealsPurchases']].sum(axis=1)
print(df.head())

     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation   relationship  71613.0        0         0   
3  6182        1984  Graduation   relationship  26646.0        1         0   
4  5324        1981         PhD   relationship  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0    4/9/2012       58       635  ...                  7             0   
1    8/3/2014       38        11  ...                  5             0   
2  21-08-2013       26       426  ...                  4             0   
3   10/2/2014       26        11  ...                  6             0   
4  19-01-2014       94       173  ...                  5             0   

   AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  Response  \
0    

### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [15]:
df['Kids'] = df[['Kidhome', 'Teenhome']].sum(axis=1)
df = df.drop(['Kidhome', 'Teenhome'], axis=1)
print(df.head())

     ID  Year_Birth   Education Marital_Status   Income Dt_Customer  Recency  \
0  5524        1957  Graduation         Single  58138.0    4/9/2012       58   
1  2174        1954  Graduation         Single  46344.0    8/3/2014       38   
2  4141        1965  Graduation   relationship  71613.0  21-08-2013       26   
3  6182        1984  Graduation   relationship  26646.0   10/2/2014       26   
4  5324        1981         PhD   relationship  58293.0  19-01-2014       94   

   MntWines  MntFruits  MntMeatProducts  ...  AcceptedCmp3  AcceptedCmp4  \
0       635         88              546  ...             0             0   
1        11          1                6  ...             0             0   
2       426         49              127  ...             0             0   
3        11          4               20  ...             0             0   
4       173         43              118  ...             0             0   

   AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  Respons

### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [16]:
df['TotalAcceptedCmp'] = df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3',
                             'AcceptedCmp4', 'AcceptedCmp5', 'Response']].sum(axis=1)
df = df.drop(['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3',
              'AcceptedCmp4', 'AcceptedCmp5', 'Response'], axis=1)
print(df.head())

     ID  Year_Birth   Education Marital_Status   Income Dt_Customer  Recency  \
0  5524        1957  Graduation         Single  58138.0    4/9/2012       58   
1  2174        1954  Graduation         Single  46344.0    8/3/2014       38   
2  4141        1965  Graduation   relationship  71613.0  21-08-2013       26   
3  6182        1984  Graduation   relationship  26646.0   10/2/2014       26   
4  5324        1981         PhD   relationship  58293.0  19-01-2014       94   

   MntWines  MntFruits  MntMeatProducts  ...  NumDealsPurchases  \
0       635         88              546  ...                  3   
1        11          1                6  ...                  2   
2       426         49              127  ...                  1   
3        11          4               20  ...                  2   
4       173         43              118  ...                  5   

   NumWebPurchases  NumCatalogPurchases  NumStorePurchases  NumWebVisitsMonth  \
0                8                 

### 11. Drop those columns which we have used above for obtaining new features

In [ ]:
already dropped

### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [18]:
current_year = pd.to_datetime('today').year
df['Age'] = current_year - df['Year_Birth']
df = df.drop('Year_Birth', axis=1)
print(df.head())

     ID   Education Marital_Status   Income Dt_Customer  Recency  MntWines  \
0  5524  Graduation         Single  58138.0    4/9/2012       58       635   
1  2174  Graduation         Single  46344.0    8/3/2014       38        11   
2  4141  Graduation   relationship  71613.0  21-08-2013       26       426   
3  6182  Graduation   relationship  26646.0   10/2/2014       26        11   
4  5324         PhD   relationship  58293.0  19-01-2014       94       173   

   MntFruits  MntMeatProducts  MntFishProducts  ...  NumWebPurchases  \
0         88              546              172  ...                8   
1          1                6                2  ...                1   
2         49              127              111  ...                8   
3          4               20               10  ...                2   
4         43              118               46  ...                5   

   NumCatalogPurchases  NumStorePurchases  NumWebVisitsMonth  Complain  \
0                   10  

### 13. Encode the categorical variables in the dataset

In [32]:
label_encoder = LabelEncoder()
df['Education'] = label_encoder.fit_transform(df['Education'])
df['Marital_Status'] = label_encoder.fit_transform(df['Marital_Status'])
print(df.head())

     ID  Year_Birth  Education  Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957          2               4  58138.0        0         0   
1  2174        1954          2               4  46344.0        1         1   
2  4141        1965          2               5  71613.0        0         0   
3  6182        1984          2               5  26646.0        1         0   
4  5324        1981          4               3  58293.0        1         0   

  Dt_Customer  Recency  MntWines  ...  NumStorePurchases  NumWebVisitsMonth  \
0    4/9/2012       58       635  ...                  4                  7   
1    8/3/2014       38        11  ...                  2                  5   
2  21-08-2013       26       426  ...                 10                  4   
3   10/2/2014       26        11  ...                  4                  6   
4  19-01-2014       94       173  ...                  6                  5   

   AcceptedCmp3  AcceptedCmp4  AcceptedCmp5  AcceptedCmp

### 14. Standardize the columns, so that values are in a particular range

In [19]:
columns_to_standardize = ['Income', 'Total_Expenses', 'Num_Total_Purchases']
scaler = StandardScaler()
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])
print(df.head())

     ID   Education Marital_Status    Income Dt_Customer  Recency  MntWines  \
0  5524  Graduation         Single  0.234063    4/9/2012       58       635   
1  2174  Graduation         Single -0.234559    8/3/2014       38        11   
2  4141  Graduation   relationship  0.769478  21-08-2013       26       426   
3  6182  Graduation   relationship -1.017239   10/2/2014       26        11   
4  5324         PhD   relationship  0.240221  19-01-2014       94       173   

   MntFruits  MntMeatProducts  MntFishProducts  ...  NumWebPurchases  \
0         88              546              172  ...                8   
1          1                6                2  ...                1   
2         49              127              111  ...                8   
3          4               20               10  ...                2   
4         43              118               46  ...                5   

   NumCatalogPurchases  NumStorePurchases  NumWebVisitsMonth  Complain  \
0                 

### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [39]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
columns_for_pca = ['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
                          'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
                          'NumWebVisitsMonth']
df = df.dropna(subset=columns_for_pca)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=columns_for_pca)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[columns_for_pca])
pca = PCA()
pca.fit(df_scaled)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
n_components_90 = np.argmax(cumulative_variance >= 0.9) + 1
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Number of components for 90% explained variance: {n_components_90}")
print(f"Explained variance with {n_components_90} components: {cumulative_variance[n_components_90-1]}")
print(f"Number of components for 95% explained variance: {n_components_95}")
print(f"Explained variance with {n_components_95} components: {cumulative_variance[n_components_95-1]}")

Number of components for 90% explained variance: 8
Explained variance with 8 components: 0.9072081565519596
Number of components for 95% explained variance: 10
Explained variance with 10 components: 0.9733824180692581


### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [24]:
columns_for_pca = ['Income', 'Total_Expenses', 'Num_Total_Purchases']
df = df.dropna(subset=columns_for_pca)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=columns_for_pca)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[columns_for_pca])
pca = PCA(n_components=2)  
pca_transformed = pca.fit_transform(df_scaled)
kmeans = KMeans(n_clusters=3)
kmeans.fit(pca_transformed)
cluster_labels = kmeans.labels_
df['Cluster'] = cluster_labels
print(df.head(

     ID   Education Marital_Status    Income Dt_Customer  Recency  MntWines  \
0  5524  Graduation         Single  0.234063    4/9/2012       58       635   
1  2174  Graduation         Single -0.234559    8/3/2014       38        11   
2  4141  Graduation   relationship  0.769478  21-08-2013       26       426   
3  6182  Graduation   relationship -1.017239   10/2/2014       26        11   
4  5324         PhD   relationship  0.240221  19-01-2014       94       173   

   MntFruits  MntMeatProducts  MntFishProducts  ...  NumCatalogPurchases  \
0         88              546              172  ...                   10   
1          1                6                2  ...                    1   
2         49              127              111  ...                    2   
3          4               20               10  ...                    0   
4         43              118               46  ...                    3   

   NumStorePurchases  NumWebVisitsMonth  Complain  Total_Expenses  \

### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [35]:
df = pd.read_csv('marketing.csv')
columns_for_clustering = ['Income', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits',
                          'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
                          'NumWebVisitsMonth']
df = df.dropna(subset=columns_for_clustering)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(subset=columns_for_clustering)
agglomerative = AgglomerativeClustering(n_clusters=3)  
cluster_labels = agglomerative.fit_predict(df[columns_for_clustering])
df['Cluster'] = cluster_labels
cluster_features = ['Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Recency',
                    'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
                    'MntSweetProducts', 'MntGoldProds', 'NumWebVisitsMonth']
for feature in cluster_features:
    analysis = df.groupby(['Cluster', feature]).size().reset_index(name='Count')
    print(f"Bivariate analysis between Cluster and {feature}:")
    print(analysis)
    print()

Bivariate analysis between Cluster and Education:
   Cluster   Education  Count
0        0    2n Cycle     87
1        0  Graduation    577
2        0      Master    184
3        0         PhD    286
4        1    2n Cycle    113
5        1       Basic     54
6        1  Graduation    538
7        1      Master    181
8        1         PhD    195
9        2  Graduation      1

Bivariate analysis between Cluster and Marital_Status:
    Cluster Marital_Status  Count
0         0         Absurd      2
1         0          Alone      1
2         0       Divorced    123
3         0        Married    446
4         0         Single    222
5         0       Together    292
6         0          Widow     48
7         1          Alone      2
8         1       Divorced    109
9         1        Married    411
10        1         Single    249
11        1       Together    280
12        1          Widow     28
13        1           YOLO      2
14        2       Together      1

Bivariate analysis 

### Visualization and Interpretation of results

-----
## Happy Learning
-----